In [2]:
import numpy as np
def UBS(b, m):
  N = len(b)
  if np.abs(np.sum(b) - m) > 1e-10:
    return False
  out = np.zeros(m, int)
  if m==1:
    out[0] = np.random.choice(a=range(N),  p=b)
    #print(out[0])
    return out

  c = 0
  i = 0
  while c < 2:
    if c < 1 and c + b[i] >= 1:
      i1 = i
    c += b[i]
    i2 = i
    i += 1
  p = np.zeros(i1+1)
  p[:i1] = b[:i1]
  p[i1]= 1-np.sum(b[:i1])
  out[0] = np.random.choice(a=range(i1+1),  p=p)
  b_new = np.zeros(N)
  #print(out[0])
  if out[0] != i1:
    for i in range(i1):
      b_new[i] = 0
    b_new[i1] = (np.sum(b[:i1+1])-1)/np.sum(b[:i1])
    for i in range(i1+1, i2):
      b_new[i] = b[i] * (1-b[i1])/((2 - np.sum(b[:i1+1])) * np.sum(b[:i1]))
    b_new[i2] = (np.sum(b[:i2+1]) - 2) + (2 - np.sum(b[:i2]))* (1-b[i1])/((2 - np.sum(b[:i1+1])) * np.sum(b[:i1]))
    for i in range(i2+1, len(b)):
      b_new[i] = b[i]
    #print("case 1", a[i1:], b_new[i1:])
    b_new = b_new/sum(b_new)*(m-1)
    out[1:] = UBS(b_new, m-1)
  if out[0] == i1:
    for i in range(i1+1):
      b_new[i] = 0
    for i in range(i1+1, i2):
      b_new[i] = b[i]/(2 - np.sum(b[:i1+1]))
    b_new[i2] = (np.sum(b[:i2+1]) - 2) + (2 - np.sum(b[:i2]))/(2 - np.sum(b[:i1+1]))
    for i in range(i2+1, len(b)):
      b_new[i] = b[i]
    #print("case 2", b_new)
    b_new = b_new/sum(b_new)*(m-1)
    out[1:] = UBS(b_new, m-1)
  return out

In [6]:
def UBS2(b_in, m):
    shuffle = np.random.permutation(b.shape[0])
    b = b_in[shuffle]
    # nudge for finite precision errors
    b[-1] = m - (np.sum(b) - b[-1])
    if m == 0:
        return np.zeros(b.shape, dtype=np.int_)
    if m == 1:
        res = np.zeros(b.shape, dtype=np.int_)
        res[np.searchsorted(np.cumsum(b), np.random.rand())] = 1
        return res
    """ Inputs a vector of probabilities b, and outputs a one-hot vector indicating which were selected """
    if len(b) == m: # all are ones
        return np.ones(b.shape)
    
    b_cumsum = np.cumsum(b)
    indexes = [0]
    prev_val = 0
    while indexes[-1] != len(b):
        next_idx = np.searchsorted(b_cumsum, prev_val + 1, side='right') - 1
        indexes.append(next_idx + 1)
        prev_val = b_cumsum[next_idx]
        
    #print(b, m, indexes)
    indexes = np.array(indexes)
    
    # we now have a list of indices to sample from
    # get their sizes and choose the one to exclude
    item_sizes = indexes[1:] - indexes[:-1]
    prob_sizes = b_cumsum[indexes[1:] - 1]
    prob_sizes[1:] -= b_cumsum[indexes[1:-1] - 1]
    p_sizes_cumsum = np.cumsum(prob_sizes)
    
    # parallelized sampling
    samples = np.random.rand(*prob_sizes.shape) * prob_sizes
    samples[1:] += p_sizes_cumsum[:-1]
    indexes_sampled = np.searchsorted(b_cumsum, samples)
    
    result = np.zeros(b.shape, dtype=np.int_)
    result[indexes_sampled] = 1
    # now go and delete some of them
    deletion_probs = 1 - prob_sizes
    
    num_to_delete = len(prob_sizes) - m
    deleted_indices = np.nonzero(UBS2(deletion_probs, num_to_delete))[0]
    #print("delind", deleted_indices)
    result[indexes_sampled[deleted_indices]] = 0
    result[shuffle] = result
    b[shuffle] = b
    # print(b, m, "res", result)
    return result

In [3]:
import numpy as np
b = [0.3, 0.3, 0.5, 0.3, 0.4, 0.5, 0.7]
a = [1, 2, 3, 4, 5, 6, 7]
print(UBS(b, 3))

[1 2 6]


In [4]:
import numpy as np
from tqdm import tqdm
b = [0.6, 0.6, 0.6, 0.6, 0.3, 0.4, 0.5, 0.4]
print(sum(b))
b_np = np.array(b)
a = [1, 2, 3, 4, 5, 6, 7]

m = 4
# ubs sampling verification
n_iters = 100000
res_total = np.zeros((len(b) + 1, ))
for n in tqdm(range(n_iters)):
    selected = np.array(UBS(b, m))
    # print(selected)
    res_total[selected] += 1
print(res_total / n_iters)

3.9999999999999996


100%|██████████| 100000/100000 [00:15<00:00, 6473.75it/s]

[0.60226 0.59986 0.59928 0.59811 0.29868 0.40395 0.49809 0.39977 0.     ]


In [5]:
n_iters = 100000
res_total = np.zeros((len(b) + 1, ))
for n in tqdm(range(n_iters)):
    res_total[:-1] += UBS2(b_np, m)
print(res_total / n_iters)

100%|██████████| 100000/100000 [00:05<00:00, 19620.77it/s]

[0.54759 0.53897 0.53638 0.53353 0.37276 0.4046  0.5263  0.40912 0.     ]


In [12]:
b_np
shuffle = np.random.permutation(b_np.shape[0])
b_s = b_np[shuffle]

In [13]:
b_s

array([0.3, 0.4, 0.5, 0.6, 0.6, 0.4, 0.6, 0.6])

In [14]:
b_s[shuffle] = b_s
print(b_s)

[0.6 0.4 0.3 0.5 0.3 0.4 0.5 0.4]


Speed test

In [132]:
cross_slice_size = 100000
m = 1000
b = np.random.rand(cross_slice_size)
b /= np.sum(b)
b *= m

print(np.sum(b))

n_iters = 10000
res_total = np.zeros((len(b) + 1, ))
for n in tqdm(range(n_iters)):
    res_total[:-1] += UBS2(b, m)
print(res_total / n_iters)

999.9999999999999


100%|██████████| 10000/10000 [00:40<00:00, 249.31it/s]

[0.0099 0.0023 0.0133 ... 0.0033 0.0116 0.    ]


In [59]:
print(b)
print((res_total[:-1]/ n_iters) - b)

[0.00815032 0.01088085 0.01494401 ... 0.01102186 0.00696576 0.00370234]
[ 0.00024968  0.00041915  0.00105599 ... -0.00922186 -0.00606576
 -0.00350234]


In [16]:
b = np.array(b)
b_cumsum = np.cumsum(b)
b_edges = np.ceil(b_cumsum)

In [111]:
UBS2(np.array(b), 3)

[0.3 0.3 0.5 0.3 0.4 0.5 0.7] 3 [0, 2, 4, 6, 7]
delind [3]


array([1, 0, 1, 0, 1, 0, 0])

In [24]:
import torch
def unbiased_sample_torch(b_in, m, device="cpu"):
    n = b_in.size(dim=0)
    
    shuffle = torch.randperm(n)
    
    b = b_in[shuffle]
    b = torch.minimum(b, torch.tensor(1 - 1e-15, device=device))
    b[-1] = m - (torch.sum(b) - b[-1]) - 1e-15
    b = torch.maximum(b, torch.tensor(0, device=device))
    
    if m == 0:
        return torch.zeros(*b.shape, dtype=torch.int, device=device)
    if m == 1:
        idx = torch.multinomial(b, 1)
        res = torch.zeros(*b.shape, dtype=torch.int, device=device)
        res[idx[0]] = 1
        return res
    """ Inputs a vector of probabilities b, and outputs a one-hot vector indicating which were selected """
    if len(b) == m: # all are ones
        return torch.ones(*b.shape, dtype=torch.int, device=device)
    
    b_cumsum = torch.cumsum(b, dim=0)
    indexes = [0]
    prev_val = 0
    while indexes[-1] != len(b):
        next_idx = torch.searchsorted(b_cumsum, prev_val + 1 + 1e-3, side='right') - 1
        # print(next_idx, len(b), b_cumsum[next_idx], b_cumsum[next_idx + 1], b_cumsum[next_idx + 2])
        indexes.append(next_idx + 1)
        prev_val = b_cumsum[next_idx]
        
    #print(b, m, indexes)
    indexes = torch.tensor(indexes, dtype=int, device=device)
    
    # we now have a list of indices to sample from
    # get their sizes and choose the one to exclude
    item_sizes = indexes[1:] - indexes[:-1]
    prob_sizes = b_cumsum[indexes[1:] - 1]
    prob_sizes[1:] -= b_cumsum[indexes[1:-1] - 1]
    p_sizes_cumsum = torch.cumsum(prob_sizes, dim=0)
    
    # parallelized sampling
    samples = torch.rand(*prob_sizes.shape, device=device) * prob_sizes
    samples[1:] += p_sizes_cumsum[:-1]
    indexes_sampled = torch.searchsorted(b_cumsum, samples)
    
    result = torch.zeros(*b.shape, dtype=torch.int, device=device)
    result[indexes_sampled] = 1
    # now go and delete some of them
    deletion_probs = 1 - prob_sizes
    
    num_to_delete = len(prob_sizes) - m
    deleted_indices = torch.nonzero(unbiased_sample_torch(deletion_probs, num_to_delete, device=device), as_tuple=True)[0]
    #print("delind", deleted_indices)
    result[indexes_sampled[deleted_indices]] = 0
    
    unshuf_order = torch.zeros_like(shuffle)
    unshuf_order[shuffle] = torch.arange(n)
    result[shuffle] = result.clone()
    # print(b, m, "res", result)
    return result

In [25]:
b = torch.Tensor([0.6, 0.6, 0.6, 0.6, 0.3, 0.4, 0.5, 0.4])
torch.sum(b)

tensor(4.)

In [26]:
m = 4
unbiased_sample_torch(b, m)

tensor([1, 0, 0, 1, 1, 0, 1, 0], dtype=torch.int32)

In [27]:
n_iters = 10000
res_total = torch.zeros(b.shape)
for n in tqdm(range(n_iters)):
    sample = unbiased_sample_torch(b, m)
    if (torch.sum(sample).numpy() != 4):
        print("AAAAA")
    res_total += unbiased_sample_torch(b, m)
print(res_total / n_iters)

 80%|████████  | 8039/10000 [00:07<00:01, 1079.40it/s]